In [ ]:
%pip install numpy==1.26.4
%pip install pandas==2.2.2
%pip install opencv-python==4.12.0.88
%pip install scikit-learn==1.4.2
%pip install matplotlib==3.8.4 seaborn==0.13.2
%pip install pillow==10.3.0
%pip install protobuf==3.20.3
%pip install ml-dtypes==0.3.1
%pip install tensorflow==2.16.1 keras==2.16.0

In [ ]:
%pip install tensorflow==2.16.1

In [ ]:
#%pip install tensorflow==2.16.1
import numpy, pandas, cv2, sklearn, tensorflow, sklearn
print("NumPy:", numpy.__version__)
print("Pandas:", pandas.__version__)
print("OpenCV:", cv2.__version__)
print("TensorFlow:", tensorflow.__version__)
print("Scikit-Learn:", sklearn.__version__)
print("CV2 ",cv2.__version__)

In [ ]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


DATA_DIR =r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection"
IMG_SIZE = 128    # 128 * 128


def find_all_images(root_dir, valid_exts={".jpg", ".jpeg", ".png"}):
    images = []
    labels = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            ext = os.path.splitext(file)[1].lower()
            if ext in valid_exts:
                img_path = os.path.join(dirpath, file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    images.append(img)

                    # Label based on folder name
                    label = 1 if "fake" in os.path.basename(dirpath).lower() else 0
                    labels.append(label)
                    print(f"Processed {img_path} with label {label}")

    return np.array(images), np.array(labels)

X, y = find_all_images(DATA_DIR)
print(X)
print("******************************************")
print(y)
X = X / 255.0  # Normalize pixel values to [0, 1]
X_flat = X.reshape(X.shape[0], -1)  # Flatten images into 1D array

X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

test_image = cv2.imread("C:\\Users\\pinka\\Digicrome\\fake-2.jpg")
test_image = cv2.resize(test_image, (IMG_SIZE, IMG_SIZE))
test_image = test_image / 255.0  # Normalize
test_image_flat = test_image.reshape(1, -1)  # Flatten

print("Test Image Prediction:", model.predict(test_image_flat))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
import numpy as np

IMG_SIZE = 128
BATCH_SIZE = 32

train_dir = r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection\\train-20250112T065955Z-001\\train"
test_dir = r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection\\test-20250112T065939Z-001\\test"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# BUlild CNN Model

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# TRain teh Model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Predict an Image

def predict_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]
    return "Fake" if prediction > 0.5 else "Real"

print(predict_image(r"C:\\Users\\pinka\\Digicrome\\fake-2.jpg"))
print(predict_image(r"C:\\Users\\pinka\\Digicrome\\real-2.jpeg"))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np

IMG_SIZE = 299  # InceptionV3 expects 299x299 input
BATCH_SIZE = 32

# Load and Process Data 
train_dir = r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection\\train-20250112T065955Z-001\\train"
test_dir = r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection\\test-20250112T065939Z-001\\test"

train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)



base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(train_data, epochs=10, validation_data=test_data)

# 🧪 Evaluate the model
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc:.2f}")


# Make Predictions

def predict_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]
    return "Fake" if prediction > 0.5 else "Real"

print(predict_image(r"C:\\Users\\pinka\\Digicrome\\fake-2.jpg"))
print(predict_image(r"C:\\Users\\pinka\\Digicrome\\real-1.jpeg"))


In [ ]:
%pip install tensorflow==2.16.1 --use-deprecated=legacy-resolver

In [ ]:
from tensorflow import keras
print("TensorFlow Keras is working!")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np

IMG_SIZE = 299  # InceptionV3 expects 299x299 input
BATCH_SIZE = 32

# Load and Process Data 
train_dir = r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection\\train-20250112T065955Z-001\\train"
test_dir = r"C:\\Users\\pinka\\Digicrome\\DeepFake Detection\\test-20250112T065939Z-001\\test"

train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)



base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(train_data, epochs=10, validation_data=test_data)

# Make Predictions

def predict_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)[0][0]
    return "Fake" if prediction > 0.5 else "Real"

print(predict_image(r"C:\\Users\\pinka\\Digicrome\\fake-2.jpg"))
print(predict_image(r"C:\\Users\\pinka\\Digicrome\\real-2.jpeg"))
